In [1]:
import numpy as np
import json
import os
from pygel3d import hmesh, gl_display as gl
from pygel3d import jupyter_display as jd
from plotly.graph_objs import Mesh3d
import plotly.graph_objects as go
import json
import matplotlib.pyplot as plt
import copy

# PATHS

In [2]:
# path to all meshes
path_meshes = '../output/cylinders-alpha=4.0-beta=0.225-targetFVF=0.8-num_cylinders=80-mode_fiber=sheets-fiber0-frac=1.0-orientation=[0,0,1]-epsilon=0.0/rep_00-stage=3-with_cells_output/ply_2/'

# select a single mesh to use as an example
name_mesh = np.sort([n for n in os.listdir(path_meshes) if ('myelin' in n)])[2]
print('name_mesh: ', name_mesh)

idx_mesh = int(name_mesh.split('_')[1].replace('.ply', ''))
print('idx_mesh: ', idx_mesh)

path_mesh = os.path.join(path_meshes, name_mesh)

# path to the config corresponding to the selected mesh
path_conf_output = '/'.join(path_meshes.split('/')[:-2]) + '/config_output_' + path_meshes.split('_')[-1].replace('/', '.json')
print('path_conf_output: ', path_conf_output)

name_mesh:  myelin_10.ply
idx_mesh:  10
path_conf_output:  ../output/cylinders-alpha=4.0-beta=0.225-targetFVF=0.8-num_cylinders=80-mode_fiber=sheets-fiber0-frac=1.0-orientation=[0,0,1]-epsilon=0.0/rep_00-stage=3-with_cells_output/config_output_2.json


# LOAD CONFIG

In [3]:
with open(path_conf_output, 'r') as file:
    conf_output = json.load(file)
    
print(conf_output.keys())

dict_keys(['growSpeed', 'contractSpeed', 'minimumDistance', 'border', 'voxelSize', 'mapFromDiameterToDeformationFactor', 'mapFromMaxDiameterToMinDiameter', 'mapFromMaxDiameterToEllipsoidSeparation', 'axons', 'cells'])


# LOAD MESH

In [4]:
m_original = hmesh.ply_load(path_mesh)

# close holes at each end
hmesh.close_holes(m_original)
# triangulate the patches that were added to close the holes
hmesh.triangulate(m_original)

RPly: Unexpected end of file
RPly: Error reading value number 3 of 'vertex_index' of 'face' number 927


# SHOW MESH

In [5]:
# jd.set_export_mode(True)
fig = jd.display(m_original)

fig.update_layout(scene=dict(
    aspectmode='data',
    xaxis_title='x [μm]',
    yaxis_title='y [μm]',
    zaxis_title='z [μm]'
))

fig.show()

# SHOW ELLIPSOID REPRESENTATION

In [6]:
cmap = plt.get_cmap('plasma')

#### Some math: generate points on the surface of ellipsoid
u, v = np.mgrid[-np.pi:np.pi:20j, -np.pi/2:np.pi/2:20j]

q = [
    np.cos(u) * np.cos(v),
    np.sin(v),
    np.sin(u) * np.cos(v)
]

#### Get data from each ellipsoid of the axon/myelin
data = []
colors = []
    
# Loop over each axon/myelin (except that we limit the loop to only loop over the selected example)
for axon in conf_output['axons'][idx_mesh:idx_mesh+1]:
    
    colors.append(axon['color'])
    
    # Something to normalise the color scale against
    scaler_color = axon['maxDiameter']

    ellipsoid_myelinDiameters = []

    # Loop over each ellipsoid
    for ellipsoid in axon['ellipsoids']:
        
        S = np.reshape(ellipsoid['shape'], (3, 3))
    
        x = S[0, 0] * q[0] + S[0, 1] * q[1] + S[0, 2] * q[2] + ellipsoid['position'][0]
        y = S[1, 0] * q[0] + S[1, 1] * q[1] + S[1, 2] * q[2] + ellipsoid['position'][1]
        z = S[2, 0] * q[0] + S[2, 1] * q[1] + S[2, 2] * q[2] + ellipsoid['position'][2]
        
        # color according to myelinDiameter
        rgba = cmap(ellipsoid['myelinDiameter'] / scaler_color - 0.5)

        # convert to format that pyplot likes
        rgba = tuple(int(value*255) for value in rgba[:-1]) + (int(rgba[-1]),)
    
        plot_ellipsoid = Mesh3d(
            x=x.flatten(),
            y=y.flatten(),
            z=z.flatten(), 
            alphahull=0,
            colorscale='viridis',
            color=f'rgba{rgba}'
        )

        data.append(plot_ellipsoid)
    
layout = go.Layout(
    scene=dict(
        aspectmode='data',
        xaxis_title='x [μm]',
        yaxis_title='y [μm]',
        zaxis_title='z [μm]'
    ), 
    width=850,
    height=800,
)

fig = go.Figure(data=data, layout=layout)
fig.show()

# CONSTRAINED GARLAND-HECKBERT-SIMPLIFICATION OF THE MESH

Simplify the mesh to decrease the number of faces and vertices for computational efficiency during Monte Carlo diffusion simulations. To prevent inducing overlaps between meshes in the phantom, the simplified mesh is constrained by not being allowed to deviate more than $minDistance/2-delta$ from the original mesh.

In [7]:
# Mesh to be optimised
m_optimise = copy.copy(m_original)

# Get the minDistance required between the myelins of the phantom
minDistance = conf_output['minimumDistance']

# Initialise the max measured distance between the original mesh and the simplified mesh
dist_abs_max = np.inf

# Initialise the (approximate) fraction of vertices to keep when performing the mesh simplification
keep_fraction = 0.01

# Loop until the deviation between the simplified and original mesh becomes higher that the threshold
while dist_abs_max > (minDistance / 2 - 0.0005):
    
    m_simplify = hmesh.Manifold(m_optimise)

    hmesh.quadric_simplify(m_simplify, keep_fraction)
    D = hmesh.MeshDistance(m_simplify)

    dist_values = D.signed_distance(m_original.positions())
    dist_abs_max = np.max(np.abs(dist_values))
    
    print(f'dist_abs_max between m_original and m_simplify: {dist_abs_max:.4f}')
    
    keep_fraction += 0.01
    
print(f'Final keep_fraction = {keep_fraction:.2f}')

dist_abs_max between m_original and m_simplify: 2.3004
dist_abs_max between m_original and m_simplify: 2.5056
dist_abs_max between m_original and m_simplify: 1.5292
dist_abs_max between m_original and m_simplify: 1.5292
dist_abs_max between m_original and m_simplify: 0.9720
dist_abs_max between m_original and m_simplify: 0.9720
dist_abs_max between m_original and m_simplify: 0.7465
dist_abs_max between m_original and m_simplify: 0.7465
dist_abs_max between m_original and m_simplify: 0.7465
dist_abs_max between m_original and m_simplify: 0.7465
dist_abs_max between m_original and m_simplify: 0.7465
dist_abs_max between m_original and m_simplify: 0.3965
dist_abs_max between m_original and m_simplify: 0.3965
dist_abs_max between m_original and m_simplify: 0.2787
dist_abs_max between m_original and m_simplify: 0.2787
dist_abs_max between m_original and m_simplify: 0.2787
dist_abs_max between m_original and m_simplify: 0.2787
dist_abs_max between m_original and m_simplify: 0.2335
dist_abs_m

In [8]:
fig = jd.display(m_simplify, data=dist_values)

fig.update_layout(
    scene=dict(
        aspectmode='data',
        xaxis_title='x [μm]',
        yaxis_title='y [μm]',
        zaxis_title='z [μm]'
    ), 
    title='The simplified colored according to its deviation in μm from the original mesh',
    # coloraxis_colorbar_title_text = 'Deviation between simplified and original mesh [μm]',
    width=850,
    height=800,
)

fig.show()